# DISCOVERY EDA
EDA SIMPLE para entender los datos que se están pasando
NO se hacen mayores análisis, solo se analizan qué features hay en los datos, frecuencia, nivel de agregación, nulos, etc

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os
# set path root of repo
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('/')[:-1]
root_path = '/'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  /Users/joseortega/Documents/GitHub/forecasting-m5-dataset


### 1. Read raw files

In [3]:
folder_data = 'data/data_input_raw/'
df_cal = pd.read_csv(folder_data + 'calendar.csv')
df_eval = pd.read_csv(folder_data + 'sales_train_evaluation.csv')
df_price = pd.read_csv(folder_data + 'sell_prices.csv')
df_sample_output = pd.read_csv(folder_data + 'sample_submission.csv')

df_validation = pd.read_csv(folder_data + 'sales_train_validation.csv')

### 2. Discovery calender
Dataframe calendario

In [4]:
df_cal.head(10)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0


In [5]:
df_cal.tail()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


#### 2.1 Nulos

In [6]:
# porcentaje de nulos
round(100 * (df_cal.isnull().sum() / df_cal.shape[0]), 2)

date             0.00
wm_yr_wk         0.00
weekday          0.00
wday             0.00
month            0.00
year             0.00
d                0.00
event_name_1    91.77
event_type_1    91.77
event_name_2    99.75
event_type_2    99.75
snap_CA          0.00
snap_TX          0.00
snap_WI          0.00
dtype: float64

#### 2.2 Indice: wm_yr_wk
Pareciera ser un índice que muestra los valores de "semana" y "año"

Además se observa que cambia su valor cuando la semana llega a día sábado. Por lo que se cuentan las semanas desde el día sábado

In [7]:
# indice "wm_yr_wk" cantidad de indices únicos
df_cal['wm_yr_wk'].unique().shape

(282,)

In [8]:
df_cal.head(2)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0


In [9]:
# crear un dataframe auxiliar y revisar si el índice wm_yr_wk corresponde a la combinación "semana"_"año"
df_cal_aux = df_cal.copy()

# obtener valor de semana conteo normal
df_cal_aux['date'] = pd.to_datetime(df_cal_aux['date'])
df_cal_aux['week'] = df_cal_aux['date'].dt.isocalendar().week 

# obtener dia de la semana PARTIENDO EL SABADO LA SEMANA - en los datos un id parece comenzar ese dia
df_cal_aux['adjusted_date'] = df_cal_aux['date'] - pd.to_timedelta((df_cal_aux['date'].dt.dayofweek + 2) % 7, unit='D')
df_cal_aux['week_saturday'] = df_cal_aux['adjusted_date'].dt.isocalendar().week

In [10]:
# contar la cantiadad de semanas año
df_cal_aux.groupby(['week', 'year']).size().shape#.reset_index()

(283,)

In [11]:
# contar la cantiadad de semanas año partiendo a contar el sabado
df_cal_aux.groupby(['week_saturday', 'year']).size().shape#.reset_index()

(283,)

In [12]:
# indice "wm_yr_wk" cantidad de indices únicos
df_cal['wm_yr_wk'].unique().shape

(282,)

Se puede observar que el índice parte tiene un valor menos, pero casi no hay dudas que representa la combinación única de semana_año

#### 2.3 Explorar eventos

In [13]:
# saber los tipos únicos de eventos - NOMBRE EVENTO 1
df_cal['event_name_1'].unique()

array([nan, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)

In [14]:
# saber los tipos únicos de eventos - NOMBRE EVENTO 2
df_cal['event_name_2'].unique()

array([nan, 'Easter', 'Cinco De Mayo', 'OrthodoxEaster', "Father's day"],
      dtype=object)

In [15]:
# saber los tipos únicos de eventos - TIPO EVENTO 1
df_cal['event_type_1'].unique()

array([nan, 'Sporting', 'Cultural', 'National', 'Religious'], dtype=object)

In [16]:
# saber los tipos únicos de eventos - TIPO EVENTO 2
df_cal['event_type_2'].unique()

# se puede observar que los eventos 2 se agregan por si EN UN DÍA hubiera MÁS DE UN TIPO DE EVENTO

array([nan, 'Cultural', 'Religious'], dtype=object)

#### 2.4 Date / d
Explorar las columnas "date" con las fechas y la columna "d" que debe ser un contador de los distintos días que hay en los datos

In [17]:
# índices únicos de columna "date"
df_cal['date'].unique().shape

(1969,)

In [18]:
# índices únicos de columna "d" contador días
df_cal['d'].unique().shape

(1969,)

#### 2.5 Explorar snap_xx
Explorar los valores de snap_CA, snap_TX, snap_WI

In [19]:
df_cal['snap_CA'].unique()

array([0, 1])

In [20]:
df_cal['snap_TX'].sum()

650

In [21]:
df_cal['snap_CA'].sum()

650

In [22]:
df_cal['snap_WI'].sum()

650

#### 2.6 Fecha min y max en los datos calendario

In [23]:
df_cal['date'].min()

'2011-01-29'

In [24]:
df_cal['date'].max()

'2016-06-19'

In [25]:
df_cal['date'].unique().shape

(1969,)

#### 2.7 Fecha min y max en el indice "wm_yr_wk"

In [26]:
# fecha minima
df_cal['wm_yr_wk'].min()

11101

In [27]:
# fecha maxima
df_cal['wm_yr_wk'].max()

11621

In [28]:
# cantidad de fechas
df_cal['wm_yr_wk'].unique().shape

(282,)

### 3. Discovery Price

In [29]:
df_price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


#### 3.0 Valores nulos

In [30]:
df_price.isnull().sum()

store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64

#### 3.1 Store id únicos
Se puede observar que son las tiendas de las ubicaciones CA, TX, WI

Se agrega un índice, me imagino que para indicar el tipo de tienda, CA_1, CA_2, etc

In [31]:
# tiendas únicas
df_price['store_id'].unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)

In [32]:
# cantidad de observaciones para cada tienda
df_price['store_id'].value_counts()

store_id
TX_2    701214
TX_1    699796
CA_1    698412
WI_3    696094
CA_3    693990
TX_3    691112
CA_4    679025
WI_2    678171
WI_1    665912
CA_2    637395
Name: count, dtype: int64

#### 3.2 Item id únicos

In [33]:
# item únicas
df_price['item_id'].unique()

array(['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003', ...,
       'FOODS_3_825', 'FOODS_3_826', 'FOODS_3_827'], dtype=object)

In [34]:
# item únicas cantidad observaciones
df_price['item_id'].unique().shape

(3049,)

In [35]:
# cantidad de observaciones para cada objeto
df_price['item_id'].value_counts()

item_id
FOODS_3_587        2820
HOUSEHOLD_1_177    2820
HOUSEHOLD_2_283    2820
HOBBIES_1_337      2820
FOODS_1_032        2820
                   ... 
HOUSEHOLD_1_308     642
HOUSEHOLD_1_159     623
HOUSEHOLD_1_242     600
FOODS_3_296         592
FOODS_2_379         530
Name: count, Length: 3049, dtype: int64

In [36]:
# cantidad de observaciones para cada objeto con respecto al total de observaciones
100 * (df_price['item_id'].value_counts() / df_price.shape[0])

item_id
FOODS_3_587        0.041221
HOUSEHOLD_1_177    0.041221
HOUSEHOLD_2_283    0.041221
HOBBIES_1_337      0.041221
FOODS_1_032        0.041221
                     ...   
HOUSEHOLD_1_308    0.009384
HOUSEHOLD_1_159    0.009107
HOUSEHOLD_1_242    0.008770
FOODS_3_296        0.008654
FOODS_2_379        0.007747
Name: count, Length: 3049, dtype: float64

#### 3.3 indice fecha wm_yr_wk - fecha minima y maxima - SEMANA/YEAR
Fecha en el dataframe de precios

Se puede observar que la información está a nivel semana. Mientras que los otros datasets están en frecuencia diaria

In [37]:
print('fecha dataframe precios')
print('fecha minima: ', df_price['wm_yr_wk'].min())
print('fecha maxima: ', df_price['wm_yr_wk'].max())
print('cantidad fechas únicas: ', df_price['wm_yr_wk'].unique().shape)

fecha dataframe precios
fecha minima:  11101
fecha maxima:  11621
cantidad fechas únicas:  (282,)


In [38]:
# mostrar fechas del dataframe calendario 
print('fecha dataframe CALENDARIO')
print('fecha minima: ', df_cal['wm_yr_wk'].min())
print('fecha maxima: ', df_cal['wm_yr_wk'].max())
print('cantidad fechas únicas: ', df_cal['wm_yr_wk'].unique().shape)

fecha dataframe CALENDARIO
fecha minima:  11101
fecha maxima:  11621
cantidad fechas únicas:  (282,)


Se puede observar que los valores mínimo y maximo en los datos son los mismos

#### 3.4 Valores de precios

In [39]:
# mostrar info de un item
mask = (df_price['store_id'] == 'CA_1') & (df_price['item_id'] == 'HOBBIES_1_001')
df_aux = df_price[mask]
df_aux

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
149,CA_1,HOBBIES_1_001,11617,8.38
150,CA_1,HOBBIES_1_001,11618,8.38
151,CA_1,HOBBIES_1_001,11619,8.38
152,CA_1,HOBBIES_1_001,11620,8.38


In [40]:
# se puede ver que el precio del producto varia a lo largo de las semanas
df_aux['sell_price'].describe()

count    154.000000
mean       8.285714
std        0.152139
min        8.260000
25%        8.260000
50%        8.260000
75%        8.260000
max        9.580000
Name: sell_price, dtype: float64

In [41]:
# valores únicos de los precios
df_aux['sell_price'].unique()

array([9.58, 8.26, 8.38])

### 4. Dataframe con las ventas

In [42]:
df_eval.shape

(30490, 1947)

#### 4.0 Valores nulos

In [43]:
df_eval.isnull().sum().sum()

0

#### 4.1 store ID únicas

In [44]:
# store id unicas en dataframe ventas
df_eval['store_id'].unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)

In [45]:
# store id unicas en dataframe precios
df_price['store_id'].unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)

#### 4.2 item_id valores únicos

In [46]:
df_eval['item_id'].unique()

array(['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003', ...,
       'FOODS_3_825', 'FOODS_3_826', 'FOODS_3_827'], dtype=object)

In [47]:
# cantidad de item_id únicos en DATAFRAME VENTAS
df_eval['item_id'].unique().shape

(3049,)

In [48]:
# cantidad de item_id unicos en dataframe precios
df_price['item_id'].unique().shape

(3049,)

#### 4.3 cat_id valores únicos

In [49]:
df_eval['cat_id'].unique()

array(['HOBBIES', 'HOUSEHOLD', 'FOODS'], dtype=object)

In [50]:
df_eval['cat_id'].unique().shape

(3,)

In [51]:
# cantidad de productos en cada cat_id
count = 0
for cat_id in df_eval['cat_id'].unique().tolist():
    df_aux = df_eval[df_eval['cat_id'] == cat_id]
    count_item_id_distintos = df_aux['item_id'].unique().shape[0]
    print(f'Para la categoría: {cat_id} hay {count_item_id_distintos} items distintos')
    count += count_item_id_distintos

Para la categoría: HOBBIES hay 565 items distintos
Para la categoría: HOUSEHOLD hay 1047 items distintos
Para la categoría: FOODS hay 1437 items distintos


In [52]:
# la suma de todos item_id de cada categoría es igual a la cantidad de item_id visto en el total del dataframe
count

3049

In [53]:
# cantidad de item_id unicos en dataframe precios
df_price['item_id'].unique().shape

(3049,)

#### 4.4 "dept_id" valores únicos. 
Esta feature está solo en el dataframe de ventas

In [54]:
df_eval['dept_id'].unique()

array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
       'FOODS_2', 'FOODS_3'], dtype=object)

In [55]:
df_eval['dept_id'].unique().shape

(7,)

In [56]:
# para cada departamento, cuantos item_id distintos hay
count = 0
for dept_id in df_eval['dept_id'].unique().tolist():
    df_aux = df_eval[df_eval['dept_id'] == dept_id]
    count_item_id_distintos = df_aux['item_id'].unique().shape[0]
    print(f'Para el departamento: {dept_id} hay {count_item_id_distintos} items distintos')
    count += count_item_id_distintos

Para el departamento: HOBBIES_1 hay 416 items distintos
Para el departamento: HOBBIES_2 hay 149 items distintos
Para el departamento: HOUSEHOLD_1 hay 532 items distintos
Para el departamento: HOUSEHOLD_2 hay 515 items distintos
Para el departamento: FOODS_1 hay 216 items distintos
Para el departamento: FOODS_2 hay 398 items distintos
Para el departamento: FOODS_3 hay 823 items distintos


In [57]:
# suma de item_id distintos en cada dept_id
count

3049

In [58]:
# cantidad de item_id distintos en total
df_eval['item_id'].unique().shape

(3049,)

Lo más probable es que cada depth id tenga ciertos item_id distintos. Los cuales los item_id no se repiten en los depth_id ya que la suma cuadró exacta

#### 4.5 state_id valores únicos

In [59]:
df_eval['state_id'].unique()

array(['CA', 'TX', 'WI'], dtype=object)

In [60]:
# cantidad de productos en cada state_id
count = 0
for state_id in df_eval['state_id'].unique().tolist():
    df_aux = df_eval[df_eval['state_id'] == state_id]
    count_item_id_distintos = df_aux['item_id'].unique().shape[0]
    print(f'Para el state: {state_id} hay {count_item_id_distintos} items distintos')
    count += count_item_id_distintos

Para el state: CA hay 3049 items distintos
Para el state: TX hay 3049 items distintos
Para el state: WI hay 3049 items distintos


Aquí se puede ver que para cada estado, están TODOS los productos

#### 4.6 Días de datos
Ver cuantos días de datos hay y comparar con el df calendario. Recordar que dataframe precios está en semanas

In [61]:
# minimo día. Es una columna en los datos
df_eval['d_1'].head(3)

0    0
1    0
2    0
Name: d_1, dtype: int64

In [62]:
# maximo de día. Es una columna en los datos
df_eval['d_1941'].head(3)

0    1
1    0
2    1
Name: d_1941, dtype: int64

In [63]:
# en el dataframe calendario. Día mínimo
df_cal['d'].min()

'd_1'

In [64]:
# en el dataframe calendario. Día máximo
# df_cal['d'].max()
df_cal.tail(1)['d'].values

array(['d_1969'], dtype=object)

##### Se puede observar que el calendario tiene más días (d_1969) que el dataframe de ventas (d_1941)

In [65]:
# hay 28 días de calendario que no están en los datos. Los 28 días que se tienen que hacer el forecast a futuro
1969 - 1941

28

#### 4.7 Semanas de datos.
Ahora comparar las semanas, esto para revisar si el dataframe de los precios también tiene 4 semanas más de datos que los datos de ventas. Utilizar el id_semana_año porque los precios están en frecuencia granularidad semanal

In [66]:
# mostrar fechas semana del dataframe precios
print('fecha dataframe precios')
print('fecha minima: ', df_price['wm_yr_wk'].min())
print('fecha maxima: ', df_price['wm_yr_wk'].max())
print('cantidad fechas únicas: ', df_price['wm_yr_wk'].unique().shape)

# mostrar fechas semana del dataframe calendario 
print('fecha dataframe CALENDARIO')
print('fecha minima: ', df_cal['wm_yr_wk'].min())
print('fecha maxima: ', df_cal['wm_yr_wk'].max())
print('cantidad fechas únicas: ', df_cal['wm_yr_wk'].unique().shape)

fecha dataframe precios
fecha minima:  11101
fecha maxima:  11621
cantidad fechas únicas:  (282,)
fecha dataframe CALENDARIO
fecha minima:  11101
fecha maxima:  11621
cantidad fechas únicas:  (282,)


In [67]:
# mostrar fechas semanas del dataframe de ventas
# SE PUEDE VER QUE EL DATAFRAME DE VENTAS NO CONTIENE INFO DE LAS SEMANAS
# LA FORMA DE OBTENER ESOS DATOS ES CRUZANDO LA INFO CON LOS DATOS DE CALENDARIO que contiene día y semana

In [68]:
# se puede observar que el último día que están en los datos de venta corresponde al ID wm_yr_wk con valor 11617
df_cal[df_cal['d'] == 'd_1941']['wm_yr_wk']

# el ultimo valor es 11621, por lo que faltan 4 semanas. Justamente las semanas para hacer la predicción

1940    11617
Name: wm_yr_wk, dtype: int64

#### 5. Dataset train validation
ESTE DATASET DEBE DE SER EL QUE CONTIENE LA INFO DE VENTAS DE LAS 4 SEMANAS.
ESTA INFO ES LA QUE SE DEBE DE PREDECIR USANDO LAS VENTAS DEL PASADO Y EL CONOCIMIENTO DE LAS VARIABLES EXÓGENAS (calendario y precios)

In [69]:
# tamaño df validation
df_validation.shape

(30490, 1919)

In [70]:
# tamaño df eval
df_eval.shape

(30490, 1947)

##### Se puede ver que ambos dataframes tienen la mismas filas y la cantidad de columnas que faltan son 28, correspondientes a los 28 días de forizonte de forecast

In [71]:
df_eval.shape[1] - df_validation.shape[1]

28

#### Se puede concluir
- Que el dataset con el nombre "validation" es el que se debería utilizar para entrenar
- Luego el dataset con el nombre "evaluation" tiene 28 días extra de ventas para utilizar como datos de validacion
- Luego el dataset de "precios" y "calendario" tienen 28 días más que el dataset "evaluation" por lo que me imagino que al momento de la competencia se utilizan estos 28 días para calcular el entregable. Y el dataset validation se utilizaba para entrenar y el primer delta de 28 días era el dataset de validación para tuneo de hiperparámetros, etc

### MODELAMIENTO
- Como en los 28 días adicionales que están en los datos "precios" y "calendario" no hay datos de ventas, por lo que esos datos no se utilizaran
- Para entrenamiento utilizar las fechas que están en el dataset "validation"
- Para test se utilizan los 28 días adicionales que están en el dataset "evaluation"